In [1]:
# Kenneth Reed
# Pandas HW4 - PyCity Schools

# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
    text-align: left;
}

In [3]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os
import csv

# File to Load (Remember to Change These)
school_data_to_load = os.path.join("Resources","schools_complete.csv")
student_data_to_load = os.path.join("Resources","students_complete.csv")

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load, encoding="UTF-8")
student_data = pd.read_csv(student_data_to_load, encoding="UTF-8")

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
#school_data_complete.head()

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [4]:
#Use .count() to see if there's missing information
#school_data_complete.count()

In [5]:
#Calculate the total number of schools
school_count = school_data_complete['School ID'].nunique()
#school_count

In [6]:
#Calculate the total number of students
student_count = school_data_complete['Student ID'].nunique()
#student_count

In [7]:
#Calculate the total budget
school_budget = school_data_complete[['School ID', 'school_name','budget']].copy()
total_budget = school_budget.drop_duplicates(['School ID', 'budget'], keep='first', inplace=True)
total_budget = school_budget.groupby('School ID').sum().reset_index()
total_budget = school_budget['budget'].sum()
#total_budget

In [8]:
#Calculate the average math score
math = school_data_complete[['Student ID', 'math_score']].copy()
avg_math = math['math_score'].mean()
#avg_math

In [9]:
#Calculate the average reading score
reading = school_data_complete[['Student ID', 'reading_score']].copy()
avg_reading = reading['reading_score'].mean()
#avg_reading

In [10]:
#Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall = (avg_math + avg_reading)/2
#overall

In [11]:
#Calculate the percentage of students with a passing math score (70 or greater)
mathPass = math.set_index("Student ID")
mathP = mathPass[mathPass['math_score'] >= 70].count()
mathP = mathP.values
mathPercentage = mathP / student_count * 100
#mathPercentage

In [12]:
#Calculate the percentage of students with a passing reading score (70 or greater)
readPass = reading.set_index("Student ID")
readP = readPass[readPass['reading_score'] >= 70].count()
readP = readP.values
readingPercentage = readP / student_count * 100
#readingPercentage

In [13]:
district = pd.DataFrame([[school_count, student_count, total_budget, avg_math, avg_reading, mathPercentage, readingPercentage, overall]],columns=['Total Schools','Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate'])
district["Total Students"] = district["Total Students"].astype(float).map("{:,.0f}".format)
district["Total Budget"] = district["Total Budget"].astype(float).map("${:,.2f}".format)
district["% Passing Math"] = district["% Passing Math"].astype(float).map("{:,.6f}".format)
district["% Passing Reading"] = district["% Passing Reading"].astype(float).map("{:,.6f}".format)
district

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [14]:
school_detail = student_data[['school_name', 'reading_score', 'math_score']].copy()
school_detail_group = school_detail.groupby('school_name')
#school_detail_group.describe()

In [15]:
#Average Math Score
#Average Reading Score
avgResults_group = school_detail.groupby('school_name').agg({'school_name':'count', 'math_score':'mean', 'reading_score':'mean'}).rename(columns={'school_name':'Total Students', 'math_score':'Average Math Score', 'reading_score':'Average Reading Score'})
count_group = school_detail.groupby('school_name').count()

#% Passing Math
PassingMath = school_detail[school_detail['math_score'] >= 70].groupby('school_name')['math_score'].count()
avgResults_group['% Passing Math'] = PassingMath / (avgResults_group['Total Students']) * 100

#% Passing Reading
PassingReading = school_detail[school_detail['reading_score'] >= 70].groupby('school_name')['reading_score'].count()
avgResults_group['% Passing Reading'] = PassingReading / (avgResults_group['Total Students']) * 100
#school_readP = school_readP.values

#% Passing Reading
#Overall Passing Rate (Average of the above two)
avgResults_group['Overall Passing Rate'] = (avgResults_group['Average Math Score'] + avgResults_group['Average Reading Score']) / 2

#avgResults_group

In [16]:
# Create a dataframe to hold the above results
school_details = pd.merge(school_data, avgResults_group, how="left", on=["school_name", "school_name"])
school_details = school_details.drop('School ID', 1)
school_details.set_index('school_name', inplace=True,)
school_details.index.name = ""
school_details.columns=['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']
school_details["Per Student Budget"] = (school_details["Total School Budget"] / school_details["Total Students"])#.astype(float).map("${:,.2f}".format)
school_details["Total Students"] = school_details["Total Students"].astype(float).map("{:,.0f}".format)
school_details["Total School Budget"] = school_details["Total School Budget"].astype(float).map("${:,.2f}".format)
school_details["Per Student Budget"] = school_details["Per Student Budget"].astype(float).map("${:,.2f}".format)
school_details

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
,,,,,,,,,
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,78.906068
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,78.934893
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,83.542589
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,79.112082
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,83.584128
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,83.631844
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,83.518837
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,79.041198
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,83.809133


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [28]:
topFiveoverall = school_details.copy()
topFiveoverall.sort_values("Overall Passing Rate", ascending=False, inplace=True)
topFiveoverall.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
,,,,,,,,,
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,83.942308
Wright High School,Charter,"1,800","$1,049,400.00",$583.00,83.682222,83.955000,93.333333,96.611111,83.818611
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,83.809133
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,83.633639
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,83.631844


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [29]:
bottomFiveoverall = school_details.copy()
bottomFiveoverall.sort_values("Overall Passing Rate", ascending=True, inplace=True)
bottomFiveoverall.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,78.793698
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,78.906068
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,78.924425
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,78.934893
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,79.019429


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [71]:
school_grade_detail = school_data_complete[['school_name', 'grade', 'reading_score', 'math_score']].copy()
school_grade_detail_math = school_grade_detail[['school_name', 'grade', 'math_score']].groupby(['school_name', 'grade']).mean()
school_grade_detail_math.index.name = "School Name"

school_grade_detail_math.columns=['Average Math Score']
#school_grade_detail_math["Per Student Budget"] = (school_details["Total School Budget"] / school_details["Total Students"])#.astype(float).map("${:,.2f}".format)
#school_grade_detail_math["Total Students"] = school_details["Total Students"].astype(float).map("{:,.0f}".format)
#school_grade_detail_math["Total School Budget"] = school_details["Total School Budget"].astype(float).map("${:,.2f}".format)
#school_grade_detail_math["Per Student Budget"] = school_details["Per Student Budget"].astype(float).map("${:,.2f}".format)
#school_details


school_grade_detail_math

Average Math Score
school_name           grade                    
Bailey High School    10th            76.996772
                      11th            77.515588
                      12th            76.492218
                      9th             77.083676
Cabrera High School   10th            83.154506
                      11th            82.765560
                      12th            83.277487
                      9th             83.094697
Figueroa High School  10th            76.539974
                      11th            76.884344
                      12th            77.151369
                      9th             76.403037
Ford High School      10th            77.672316
                      11th            76.918058
                      12th            76.179963
                      9th             77.361345
Griffin High School   10th            84.229064
                      11th            83.842105
                      12th            83.356164
                      9th             82.044010
Hernandez High School 10th            77.337408
                      11th            77.136029
                      12th            77.186567
                      9th             77.438495
Holden High School    10th            83.429825
                      11th            85.000000
                      12th            82.855422
                      9th             83.787402
Huang High School     10th            75.908735
                      11th            76.446602
                      12th            77.225641
                      9th             77.027251
Johnson High School   10th            76.691117
                      11th            77.491653
                      12th            76.863248
                      9th             77.187857
Pena High School      10th            83.372000
                      11th            84.328125
                      12th            84.121547
                      9th             83.625455
Rodriguez High School 10th            76.612500
                      11th            76.395626
                      12th            77.690748
                      9th             76.859966
Shelton High School   10th            82.917411
                      11th            83.383495
                      12th            83.778976
                      9th             83.420755
Thomas High School    10th            83.087886
                      11th            83.498795
                      12th            83.497041
                      9th             83.590022
Wilson High School    10th            83.724422
                      11th            83.195326
                      12th            83.035794
                      9th             83.085578
Wright High School    10th            84.010288
                      11th            83.836782
                      12th            83.644986
                      9th             83.264706

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [61]:
school_grade_detail_reading = school_grade_detail[['school_name', 'grade', 'reading_score']].groupby(['school_name', 'grade']).mean()
school_grade_detail_reading

reading_score
school_name           grade               
Bailey High School    10th       80.907183
                      11th       80.945643
                      12th       80.912451
                      9th        81.303155
Cabrera High School   10th       84.253219
                      11th       83.788382
                      12th       84.287958
                      9th        83.676136
Figueroa High School  10th       81.408912
                      11th       80.640339
                      12th       81.384863
                      9th        81.198598
Ford High School      10th       81.262712
                      11th       80.403642
                      12th       80.662338
                      9th        80.632653
Griffin High School   10th       83.706897
                      11th       84.288089
                      12th       84.013699
                      9th        83.369193
Hernandez High School 10th       80.660147
                      11th       81.396140
                      12th       80.857143
                      9th        80.866860
Holden High School    10th       83.324561
                      11th       83.815534
                      12th       84.698795
                      9th        83.677165
Huang High School     10th       81.512386
                      11th       81.417476
                      12th       80.305983
                      9th        81.290284
Johnson High School   10th       80.773431
                      11th       80.616027
                      12th       81.227564
                      9th        81.260714
Pena High School      10th       83.612000
                      11th       84.335938
                      12th       84.591160
                      9th        83.807273
Rodriguez High School 10th       80.629808
                      11th       80.864811
                      12th       80.376426
                      9th        80.993127
Shelton High School   10th       83.441964
                      11th       84.373786
                      12th       82.781671
                      9th        84.122642
Thomas High School    10th       84.254157
                      11th       83.585542
                      12th       83.831361
                      9th        83.728850
Wilson High School    10th       84.021452
                      11th       83.764608
                      12th       84.317673
                      9th        83.939778
Wright High School    10th       83.812757
                      11th       84.156322
                      12th       84.073171
                      9th        83.833333

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [17]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [18]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.